In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib
import matplotlib.pyplot as plt # for data visualization
import seaborn as sns # for statistical data visualization
%matplotlib inline
from ExKMC.Tree import Tree
from IPython.display import Image

np.random.seed(1)
import random
import sys

import sklearn
import sklearn.ensemble
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

import shap
import joblib as jbl

import lime
import lime.lime_tabular

from anchor import utils
from anchor import anchor_tabular

import json
import warnings

/tmp/ipykernel_144949/4059321248.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


In [2]:
data_yes = pd.read_csv("Dataset_99k_yes.csv")
data_no = pd.read_csv("Dataset_75k_no.csv")

sample_data_yes = data_yes.sample(n=75000)
sample_data_no = data_no.sample(n=75000)
sample_data_no = sample_data_no.rename(columns={"id": "id_"})

training_samples = pd.concat([sample_data_yes, sample_data_no], axis=0)
display(training_samples)

,id_,methodAnonymousClassesQty,methodAssignmentsQty,methodCbo,methodComparisonsQty,methodLambdasQty,methodLoc,methodLoopQty,methodMathOperationsQty,methodMaxNestedBlocks,...,methodRfc,methodStringLiteralsQty,methodSubClassesQty,methodTryCatchQty,methodUniqueWordsQty,methodVariablesQty,methodWmc,bugFixCount,refactoringsInvolved,y
1225,1225,0,0,1,0,0,7,0,0,0,...,11,1,0,0,25,0,1,70,17,1
10786,10791,0,12,4,2,0,29,1,0,5,...,10,0,0,1,30,9,8,0,0,1
43890,43918,0,3,3,0,0,5,0,0,0,...,0,0,0,0,8,0,1,0,1,1
68396,68448,0,3,6,0,0,20,0,2,3,...,8,5,0,2,23,2,4,3,5,1
49739,49771,0,0,4,0,0,10,1,0,3,...,2,0,0,0,10,0,4,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29751,683654,0,7,3,0,0,33,0,0,4,...,11,0,0,1,29,3,8,2,6,0
14797,973702,0,0,1,0,0,3,0,0,0,...,1,0,0,0,7,0,1,26,9,0
26002,1068909,0,0,1,0,0,3,0,0,0,...,1,0,0,0,3,0,1,0,0,0
14235,1044334,0,0,0,0,0,4,0,0,0,...,1,0,0,0,10,0,1,32,10,0


In [3]:
# importing training dataset
data = training_samples.set_index("id_")
display(data)

# splitting train and test
X = data.drop(['y'], axis=1)
y = data['y']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 42)

# training random forest model
forest = RandomForestClassifier(random_state=0)
forest.fit(X_train, y_train)

# getting list of feature names
feature_names = list(X_train.columns)

,methodAnonymousClassesQty,methodAssignmentsQty,methodCbo,methodComparisonsQty,methodLambdasQty,methodLoc,methodLoopQty,methodMathOperationsQty,methodMaxNestedBlocks,methodNumbersQty,...,methodRfc,methodStringLiteralsQty,methodSubClassesQty,methodTryCatchQty,methodUniqueWordsQty,methodVariablesQty,methodWmc,bugFixCount,refactoringsInvolved,y
id_,,,,,,,,,,,,,,,,,,,,,
1225,0,0,1,0,0,7,0,0,0,0,...,11,1,0,0,25,0,1,70,17,1
10791,0,12,4,2,0,29,1,0,5,1,...,10,0,0,1,30,9,8,0,0,1
43918,0,3,3,0,0,5,0,0,0,0,...,0,0,0,0,8,0,1,0,1,1
68448,0,3,6,0,0,20,0,2,3,0,...,8,5,0,2,23,2,4,3,5,1
49771,0,0,4,0,0,10,1,0,3,0,...,2,0,0,0,10,0,4,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
683654,0,7,3,0,0,33,0,0,4,0,...,11,0,0,1,29,3,8,2,6,0
973702,0,0,1,0,0,3,0,0,0,0,...,1,0,0,0,7,0,1,26,9,0
1068909,0,0,1,0,0,3,0,0,0,1,...,1,0,0,0,3,0,1,0,0,0


In [4]:
# importing cluster0 to explain
cluster0 = pd.read_csv("Cluster_00.csv")
# getting cluster ids
samples_indexes_cluster0 = cluster0['id_'].to_list()    
cluster0 = cluster0.set_index("id_")
cluster0 = cluster0.drop('cluster', axis=1)

# importing cluster2 to explain
cluster2 = pd.read_csv("Cluster_02.csv")
# getting cluster ids
samples_indexes_cluster2 = cluster2['id_'].to_list()    
cluster2 = cluster2.set_index("id_")
cluster2 = cluster2.drop('cluster', axis=1)

def separate_samples_by_category(cluster, samples_indexes):
    
    
    negative_predict, predict_50_60, predict_60_90, predict_90 = ([] for i in range(4))
    for i in range(0, len(cluster)):
        predict = forest.predict_proba(cluster.iloc[[i]])[0][1]
        if predict < 0.5:
            negative_predict.append(samples_indexes[i])
        elif predict >= 0.5 and predict < 0.6:
            predict_50_60.append(samples_indexes[i])
        elif predict >= 0.6 and predict < 0.9:
            predict_60_90.append(samples_indexes[i])
        elif predict >= 0.9:
            predict_90.append(samples_indexes[i])
    cluster = cluster.drop(negative_predict, axis=0)
    # display(cluster)
    samples_predict_50_60 = random.sample(predict_50_60, 5) if len(predict_50_60) > 4 else predict_50_60
    samples_predict_60_90 = random.sample(predict_60_90, 5) if len(predict_60_90) > 4 else predict_60_90
    samples_predict_90 = random.sample(predict_90, 5) if len(predict_90) > 4 else predict_90
    print("Negative predictions: ", len(negative_predict), "\nPredictions beetwen 0.5 and 0.6: ", len(predict_50_60), "\nPredictions beetwen 0.6 and 0.9: ", len(predict_60_90), "\nPredictions higher than 0.9: ", len(predict_90))
    return [samples_predict_50_60, samples_predict_60_90, samples_predict_90]
    
print("Cluster 0")
samples_indexes_cluster0 = separate_samples_by_category(cluster0, samples_indexes_cluster0)
print("Cluster 2")
samples_indexes_cluster2 = separate_samples_by_category(cluster2, samples_indexes_cluster2)

Cluster 0
Negative predictions:  7 
Predictions beetwen 0.5 and 0.6:  3 
Predictions beetwen 0.6 and 0.9:  23 
Predictions higher than 0.9:  7
Cluster 2
Negative predictions:  0 
Predictions beetwen 0.5 and 0.6:  2 
Predictions beetwen 0.6 and 0.9:  23 
Predictions higher than 0.9:  15


In [5]:
cluster0_samples_predict_50_60 = cluster0.loc[samples_indexes_cluster0[0]]
cluster0_samples_predict_60_90 = cluster0.loc[samples_indexes_cluster0[1]]
cluster0_samples_predict_90 = cluster0.loc[samples_indexes_cluster0[2]]

In [6]:
samples_cluster0 = pd.concat([cluster0_samples_predict_50_60, cluster0_samples_predict_60_90, cluster0_samples_predict_90])
display(samples_cluster0)
simplified_samples_indexes_cluster0 = np.concatenate(samples_indexes_cluster0)

,methodAnonymousClassesQty,methodAssignmentsQty,methodCbo,methodComparisonsQty,methodLambdasQty,methodLoc,methodLoopQty,methodMathOperationsQty,methodMaxNestedBlocks,methodNumbersQty,...,methodReturnQty,methodRfc,methodStringLiteralsQty,methodSubClassesQty,methodTryCatchQty,methodUniqueWordsQty,methodVariablesQty,methodWmc,bugFixCount,refactoringsInvolved
id_,,,,,,,,,,,,,,,,,,,,,
57089,0,0,2,0,0,7,0,2,0,3,...,0,3,4,0,0,26,0,3,13,10
46888,0,2,4,2,0,8,0,0,0,0,...,0,0,3,0,0,19,0,4,2,2
20374,0,3,2,1,0,12,0,1,1,3,...,0,4,0,0,0,13,1,4,0,0
18374,0,3,3,0,0,11,1,0,2,3,...,2,4,0,0,0,10,1,3,4,8
58800,2,4,11,0,0,83,0,1,1,0,...,2,2,1,0,0,46,4,3,0,0
97600,0,4,8,1,0,17,0,0,1,1,...,0,7,0,0,0,26,4,2,3,2
87901,0,8,1,0,0,11,0,3,1,0,...,0,4,12,0,0,29,8,1,47,20
60792,0,5,5,0,0,11,0,3,1,4,...,1,5,0,0,0,11,2,1,1,0
62297,0,10,10,1,0,30,0,0,1,0,...,1,8,2,0,0,29,6,6,3,7


In [7]:
# Setting up SHAP
try:
    with open("./" + 'shap_explainer', 'rb') as f:
        explainer_shap = jbl.load(f)
except:
    explainer_shap = shap.TreeExplainer(forest)
    with open("./" + 'shap_explainer', 'wb') as f:
        jbl.dump(explainer_shap, f)

# Setting up LIME
explainer_lime = lime.lime_tabular.LimeTabularExplainer(X_train.values, feature_names=X_train.columns.values.tolist(), class_names=[0, 1], verbose=True, mode='classification',  discretize_continuous=True)

# Setting up Anchors
explainer_anchors = anchor_tabular.AnchorTabularExplainer(
    [0, 1],
    X_train.columns.values.tolist(),
    X_train.values,
    {})

In [16]:
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

# function to help sorting features
def compare_shap_feature_weights(feature_value):
    # highest weights first
    return -feature_value['feature_weight']

# export shap features' explanations
def export_shap_exp(row, feature_names, shap_values):
    # get shap values to refactor instance
    shap_values_to_refactor = shap_values[1]
    shap_output = dict()
    
    features_exp = []
    for i in range(0, len(feature_names)):
        # shap_values has positive values (to refactor) and negative ones (not to refactor)
        if shap_values_to_refactor[i] > 0:
            f = dict()
            feature_name = feature_names[i]
            f['feature_name'] = feature_name
            f['feature_value'] = int(row[feature_name])
            f['feature_weight'] = shap_values_to_refactor[i]
            f['feature_ranges'] = None
            features_exp.append(f)
    # sort features by feature weight
    sorted_features_exp = sorted(features_exp, key=compare_shap_feature_weights)
    # append feature ranking after sort
    rank = 1
    for f in sorted_features_exp:
        f['feature_rank'] = rank
        rank = rank + 1
    shap_output['features'] = sorted_features_exp
    return shap_output

# export lime features' explanations
def export_lime_exp(row, feature_names, exp_lime):
    lime_output = dict()
    # general instance indices
    lime_output['intercept'] = exp_lime.intercept[1]
    lime_output['local_prediction'] = exp_lime.local_pred[0]
    lime_features = exp_lime.as_list()
    # features' values
    features_exp = []
    rank = 1
    for value in lime_features:
        # value[1] (feature_weight) has positive values (to refactor) and negative ones (not to refactor)
        if value[1] > 0:
            f = dict()
            # extract feature name from feature ranges string
            any((feature_name := substring) in value[0] for substring in feature_names)
            f['feature_name'] = feature_name
            f['feature_value'] = int(row[feature_name])
            f['feature_weight'] = value[1]
            f['feature_ranges'] = value[0]
            f['feature_rank'] = rank # feature's order of priority in explainer's result
            features_exp.append(f)
            rank = rank + 1 # ranks the positive features
    lime_output['features'] = features_exp
    return lime_output

# export anchors features' explanations
def export_anchors_exp(row, feature_names, anchors_exp):
    anchors_output = dict()
    # general instance indices
    anchors_output['precision'] = anchors_exp.precision()
    anchors_output['coverage'] = anchors_exp.coverage()
    # features' values
    features_exp = []
    rank = 1
    for name in anchors_exp.names():
        f = dict()
        # extract feature name from anchors' names string
        any((feature_name := substring) in str(name) for substring in feature_names)
        f['feature_name'] = feature_name
        f['feature_value'] = int(row[feature_name])
        f['feature_weight'] = None
        f['feature_ranges'] = name
        f['feature_rank'] = rank # feature's order of priority in explainer's result
        features_exp.append(f)
        rank = rank + 1
    anchors_output['features'] = features_exp
    return anchors_output

def run_and_export_explainers(samples, i):
    feature_names = list(samples.columns)
    
    #SHAP
    row = samples.iloc[i]
    shap_values = explainer_shap.shap_values(row)
    shap_output = export_shap_exp(row, feature_names, shap_values)

    #LIME
    exp_lime = explainer_lime.explain_instance(samples.values[i], forest.predict_proba, num_features=8)
    lime_output = export_lime_exp(row, feature_names, exp_lime)

    #ANCHORS
    exp_anchors = explainer_anchors.explain_instance(samples.values[i], forest.predict, threshold=0.95)
    anchors_output = export_anchors_exp(row, feature_names, exp_anchors)
    
    return [shap_output, lime_output, anchors_output]

def export_json_explanations(samples, indexes):
    explanations = dict()
    for i in range(0, len(samples)):
        json_output = dict()
        # gives values of random forest predictions
        predict = forest.predict_proba(samples.iloc[[i]])[0]
        json_output['forest_prediction'] = {'not to refactor': predict[0], 'refactor': predict[1]}
        # calls funtions that run and export shap, lime and anchors explanations
        json_output['shap'], json_output['lime'], json_output['anchors'] = run_and_export_explainers(samples, i)
        # puts explanations in each instance index
        explanations[indexes[i]] = json_output
        #export explanations to json file
    with open("explanations_samples.json", "w") as outfile:
        json.dump(explanations, outfile)
    return explanations

In [17]:
json_output_exp = export_json_explanations(samples_cluster0, simplified_samples_indexes_cluster0)
#pretty print for json
print(json.dumps(json_output_exp, indent=4))

Intercept 0.5045968500351493
Prediction_local [0.51214689]
Right: 0.51
Intercept 0.46034516580251844
Prediction_local [0.57742975]
Right: 0.59
Intercept 0.3877309953158127
Prediction_local [0.62300084]
Right: 0.58
Intercept 0.5065847579563105
Prediction_local [0.62382599]
Right: 0.63
Intercept 0.4538926086462499
Prediction_local [0.61743803]
Right: 0.69
Intercept 0.4487838580852905
Prediction_local [0.69248528]
Right: 0.67
Intercept 0.53676376321417
Prediction_local [0.37938326]
Right: 0.71
Intercept 0.4618322373613285
Prediction_local [0.64036042]
Right: 0.82
Intercept 0.45379865395947405
Prediction_local [0.70583205]
Right: 0.92
Intercept 0.5324508323094368
Prediction_local [0.58770326]
Right: 0.91
Intercept 0.41517368307666525
Prediction_local [0.70804442]
Right: 0.96
Intercept 0.42021106303329187
Prediction_local [0.71055638]
Right: 0.95
Intercept 0.4420910310655672
Prediction_local [0.74069009]
Right: 0.92


TypeError: keys must be str, int, float, bool or None, not int64

In [23]:
print(json_output_exp)

{57089: {'forest_prediction': {'not to refactor': 0.49, 'refactor': 0.51}, 'shap': {'features': [{'feature_name': 'methodUniqueWordsQty', 'feature_value': 26, 'feature_weight': 0.11943294673452497, 'feature_ranges': None, 'feature_rank': 1}, {'feature_name': 'refactoringsInvolved', 'feature_value': 10, 'feature_weight': 0.028117718917425732, 'feature_ranges': None, 'feature_rank': 2}, {'feature_name': 'methodWmc', 'feature_value': 3, 'feature_weight': 0.02766035594743198, 'feature_ranges': None, 'feature_rank': 3}, {'feature_name': 'methodMathOperationsQty', 'feature_value': 2, 'feature_weight': 0.02655349966856471, 'feature_ranges': None, 'feature_rank': 4}, {'feature_name': 'methodNumbersQty', 'feature_value': 3, 'feature_weight': 0.02395352220817281, 'feature_ranges': None, 'feature_rank': 5}, {'feature_name': 'methodParametersQty', 'feature_value': 1, 'feature_weight': 0.018798843534847415, 'feature_ranges': None, 'feature_rank': 6}, {'feature_name': 'methodCbo', 'feature_value': 2

In [143]:
def extract_feature_names(json_features):
    extracted_feature_names = []
    for f in json_features:
        extracted_feature_names.append(f['feature_name'])
    return extracted_feature_names

def get_info_by_feature_name(exp_f, feature_name):
    feature_obj = next(x for x in exp_f if x["feature_name"] == feature_name)
    return feature_obj['feature_value'], feature_obj['feature_weight'], feature_obj['feature_ranges'], feature_obj['feature_rank']

def compare_feature_weights(feature_value):
    # order = ['rank_anchors', 'rank_shap', 'rank_lime']
    # rank_order = []

    # for o in order:
    #     if o in feature_value
    #         rank_order.append(feature_value[o])
        
    if 'rank_shap' in feature_value and 'rank_lime' in feature_value and 'rank_anchors' in feature_value:
        return feature_value['rank_anchors'], feature_value['rank_shap'], feature_value['rank_lime']
    elif 'rank_shap' in feature_value and 'rank_lime' in feature_value:
        return feature_value['rank_shap'], feature_value['rank_lime']
    elif 'rank_shap' in feature_value and 'rank_anchors' in feature_value:
        return feature_value['rank_anchors'], feature_value['rank_shap']
    elif 'rank_lime' in feature_value and 'rank_anchors' in feature_value:
        return feature_value['rank_anchors'], feature_value['rank_lime']

def intersect_feature_explanations(feature_names, instance):
    shap_f = instance['shap']['features']
    lime_f = instance['lime']['features']
    anchors_f = instance['anchors']['features']

    shap_f_names = extract_feature_names(shap_f)
    lime_f_names = extract_feature_names(lime_f)
    anchors_f_names = extract_feature_names(anchors_f)

    intersec_3, intersec_shap_lime, intersec_shap_anchors, intersec_lime_anchors = ([] for i in range(4))
    compiled_intersec = dict()
    for f in feature_names:
        # for each configuration of intersection, gets feature name and explainers ranking
        # when finds feature in shap, lime and anchors
        if f in shap_f_names and f in lime_f_names and f in anchors_f_names:
            feature_summary = dict()
            feature_summary['feature_name'] = f
            # get info from explainers' features
            feature_value, _, anchors_ranges, anchors_rank = get_info_by_feature_name(anchors_f, f)
            _, shap_weight, _, shap_rank = get_info_by_feature_name(shap_f, f)
            _, lime_weight, lime_ranges, lime_rank = get_info_by_feature_name(lime_f, f)
            # feature value
            feature_summary['feature_value'] = feature_value
            # puts in dictionary
            feature_summary['rank_anchors'] = anchors_rank
            feature_summary['rank_shap'] = shap_rank
            feature_summary['rank_lime'] = lime_rank
            feature_summary['weight_shap'] = shap_weight
            feature_summary['weight_lime'] = lime_weight
            feature_summary['range_anchors'] = anchors_ranges
            feature_summary['range_lime'] = lime_ranges
            intersec_3.append(feature_summary)
        # when finds feature in shap and lime
        elif f in shap_f_names and f in lime_f_names:
            feature_summary = dict()
            feature_summary['feature_name'] = f
            # get info from explainers' features
            feature_value, shap_weight, _, shap_rank = get_info_by_feature_name(shap_f, f)
            _, lime_weight, lime_ranges, lime_rank = get_info_by_feature_name(lime_f, f)
            # feature value
            feature_summary['feature_value'] = feature_value
            # puts in dictionary
            feature_summary['rank_shap'] = shap_rank
            feature_summary['rank_lime'] = lime_rank
            feature_summary['weight_shap'] = shap_weight
            feature_summary['weight_lime'] = lime_weight
            feature_summary['range_lime'] = lime_ranges
            intersec_shap_lime.append(feature_summary)
        # when finds feature in shap and anchors
        elif f in shap_f_names and f in anchors_f_names:
            feature_summary = dict()
            feature_summary['feature_name'] = f
            # get info from explainers' features
            feature_value, _, anchors_ranges, anchors_rank = get_info_by_feature_name(anchors_f, f)
            _, shap_weight, _, shap_rank = get_info_by_feature_name(shap_f, f)
            # feature value
            feature_summary['feature_value'] = feature_value
            # puts in dictionary
            feature_summary['rank_anchors'] = anchors_rank
            feature_summary['rank_shap'] = shap_rank
            feature_summary['weight_shap'] = shap_weight
            feature_summary['range_anchors'] = anchors_ranges
            intersec_shap_anchors.append(feature_summary)
        # when finds feature in lime and anchors
        elif f in lime_f_names and f in anchors_f_names:
            feature_summary = dict()
            feature_summary['feature_name'] = f
            # get info from explainers' features
            feature_value, _, anchors_ranges, anchors_rank = get_info_by_feature_name(anchors_f, f)
            _, lime_weight, lime_ranges, lime_rank = get_info_by_feature_name(lime_f, f)
            # feature value
            feature_summary['feature_value'] = feature_value
            # puts in dictionary
            feature_summary['rank_anchors'] = anchors_rank
            feature_summary['rank_lime'] = lime_rank
            feature_summary['weight_lime'] = lime_weight
            feature_summary['range_anchors'] = anchors_ranges
            feature_summary['range_lime'] = lime_ranges
            intersec_lime_anchors.append(feature_summary)

    # order each list by importance (1st anchors, 2nd shap, 3rd lime)
    compiled_intersec['intersec_all'] = sorted(intersec_3, key=compare_feature_weights)
    compiled_intersec['intersec_shap_lime'] = sorted(intersec_shap_lime, key=compare_feature_weights)
    compiled_intersec['intersec_shap_anchors'] = sorted(intersec_shap_anchors, key=compare_feature_weights)
    compiled_intersec['intersec_lime_anchors'] = sorted(intersec_lime_anchors, key=compare_feature_weights)
    
    # print(json.dumps(compiled_intersec, indent=4))
    return compiled_intersec

# 1st: get interseccions between all the 3 explainers
# 2nd: get interseccions between 2 explainers
#      decision between explainers will have
#      priority -> 1 - anchors, 2 - shap, 3 - lime
def generate_top_n_features_v1(feature_names, json_out, n_f):
    final_explanations = dict()
    # run each instance
    for i in json_out:
        intersections = intersect_feature_explanations(feature_names, json_out[i])
        # 1st step of algorithm
        # get intersections between all explainers
        final_n_features = intersections['intersec_all'][0:n_f]
        # verify how many features misses to complete the x top features
        missing_features = n_f - len(final_n_features)
        
        if missing_features:
            # get x reamining features from each of intersections between 2 explainers
            i_shap_lime_f = intersections['intersec_shap_lime'][0:missing_features]
            i_shap_anchors_f = intersections['intersec_shap_anchors'][0:missing_features]
            i_lime_anchors_f = intersections['intersec_lime_anchors'][0:missing_features]
            # concat lists
            remaining_features = i_shap_lime_f+i_shap_anchors_f+i_lime_anchors_f
            # 2st step of algorithm
            # sort concatenated list
            sorted_remaining_features = sorted(remaining_features, key=compare_feature_weights)
            # concat features selected
            final_n_features = final_n_features + sorted_remaining_features[0:missing_features]

        instance_final_exp = dict()
        instance_final_exp['forest_prediction_to_refactor'] = json_out[i]['forest_prediction']['refactor']
        top_n_features_string = 'top_'+str(n_f)+'_features'
        instance_final_exp[top_n_features_string] = final_n_features
        
        final_explanations[i] = instance_final_exp

    #export explanations to json file
    file_name = 'top_'+str(n_f)+'_features_explanations.json'
    with open(file_name, "w") as outfile:
        json.dump(final_explanations, outfile)
        
    return final_explanations

final_explanations = generate_top_n_features_v1(feature_names, json_output_exp, 5)
print(json.dumps(final_explanations, indent=4))

TypeError: keys must be str, int, float, bool or None, not int64